In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
# !ls /home/aistudio/data

In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
# !ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
# !mkdir /home/aistudio/external-libraries
# !pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
# import sys 
# sys.path.append('/home/aistudio/external-libraries')

In [1]:
!pip install paddlepaddle-gpu
import paddle as paddle
import os
# Initialize PaddlePaddle.
with_gpu = os.getenv('WITH_GPU', '1') != '0'
import paddle
print("paddle " + paddle.__version__)
device = paddle.set_device("gpu")  # 指定设备
print("using {} device.".format(device))

Looking in indexes: https://mirror.baidu.com/pypi/simple/
paddle 2.0.1
using CUDAPlace(0) device.


In [2]:

import zipfile
with zipfile.ZipFile("data/data35095/train.zip") as zf:
   zf.extractall()
! mv train data/dataset

import os
from shutil import copy, rmtree
import random


def mk_file(file_path: str):
    if os.path.exists(file_path):
        # 如果文件夹存在，则先删除原文件夹在重新创建
        rmtree(file_path)
    os.makedirs(file_path)

# 保证随机可复现
random.seed(0)

# 将数据集中10%的数据划分到验证集中
split_rate = 0.1

# 指向你解压后的flower_photos文件夹
# cwd = os.getcwd()
data_root = "data/"
origin_garbage_path = "data/dataset"
assert os.path.exists(origin_garbage_path)
garbage_class = [cla for cla in os.listdir(origin_garbage_path)
                if os.path.isdir(os.path.join(origin_garbage_path, cla))]

# 建立保存训练集的文件夹
train_root = os.path.join(data_root, "train")
mk_file(train_root)
for cla in garbage_class:
    # 建立每个类别对应的文件夹
    mk_file(os.path.join(train_root, cla))

# 建立保存验证集的文件夹
val_root = os.path.join(data_root, "val")
mk_file(val_root)
for cla in garbage_class:
    # 建立每个类别对应的文件夹
    mk_file(os.path.join(val_root, cla))

for cla in garbage_class:
    cla_path = os.path.join(origin_garbage_path, cla)
    images = os.listdir(cla_path)
    num = len(images)
    # 随机采样验证集的索引
    eval_index = random.sample(images, k=int(num*split_rate))
    for index, image in enumerate(images):
        if image in eval_index:
            # 将分配至验证集中的文件复制到相应目录
            image_path = os.path.join(cla_path, image)
            new_path = os.path.join(val_root, cla)
            copy(image_path, new_path)
        else:
            # 将分配至训练集中的文件复制到相应目录
            image_path = os.path.join(cla_path, image)
            new_path = os.path.join(train_root, cla)
            copy(image_path, new_path)
        print("\r[{}] processing [{}/{}]".format(cla, index+1, num), end="")  # processing bar
    print()


[18] processing [352/352]
[10] processing [377/377]
[35] processing [341/341]
[21] processing [647/647]
[27] processing [526/526]
[7] processing [352/352]
[39] processing [427/427]
[33] processing [312/312]
[19] processing [302/302]
[15] processing [409/409]
[6] processing [385/385]
[38] processing [381/381]
[13] processing [399/399]
[37] processing [312/312]
[16] processing [342/342]
[20] processing [216/216]
[30] processing [311/311]
[0] processing [232/232]
[17] processing [299/299]
[8] processing [370/370]
[2] processing [269/269]
[5] processing [279/279]
[31] processing [436/436]
[22] processing [365/365]
[36] processing [255/255]
[9] processing [379/379]
[3] processing [75/75]
[4] processing [377/377]
[32] processing [270/270]
[1] processing [360/360]
[29] processing [406/406]
[12] processing [321/321]
[14] processing [347/347]
[11] processing [726/726]
[23] processing [299/299]
[25] processing [540/540]
[34] processing [385/385]
[28] processing [372/372]
[24] processing [308/308

In [3]:
import paddle.nn as nn
import paddle
import paddle.nn.functional as F


class GoogLeNet(nn.Layer):
    def __init__(self, num_classes=1000, aux_logits=True):
        # 类别个数， 是否使用辅助分类器
        super(GoogLeNet, self).__init__()
        self.aux_logits = aux_logits

        self.conv1 = BasicConv2D(3, 64, kernel_size=7, stride=2, padding=3, weight_attr=nn.initializer.KaimingNormal())  # 输入
        # (224-7+6)/2+1=112
        # 3*224*224 -> 64*112*112
        self.maxpool1 = nn.MaxPool2D(3, stride=2, ceil_mode=True)  # 小数的话向上取整
        # (112-3)/2+1=56
        # 64*56*56

        self.conv2 = BasicConv2D(64, 64, kernel_size=1, weight_attr=nn.initializer.KaimingNormal())
        # 64*56*56
        self.conv3 = BasicConv2D(64, 192, kernel_size=3, padding=1, weight_attr=nn.initializer.KaimingNormal())
        # 192*56*56
        self.maxpool2 = nn.MaxPool2D(3, stride=2, ceil_mode=True)
        # 192*28*28

        self.inception3a = Inception(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = Inception(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2D(3, stride=2, ceil_mode=True)

        self.inception4a = Inception(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = Inception(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = Inception(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = Inception(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = Inception(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2D(3, stride=2, ceil_mode=True)

        self.inception5a = Inception(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = Inception(832, 384, 192, 384, 48, 128, 128)

        if self.aux_logits:
            self.aux1 = InceptionAux(512, num_classes)  # 输入为4a的输出
            self.aux2 = InceptionAux(528, num_classes)  # 输入为4d输出

        self.avgpool = nn.AdaptiveAvgPool2D((1, 1))  # 自适应的，参数为输出矩阵的高与宽
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(1024, num_classes, weight_attr=nn.initializer.KaimingNormal())

    def forward(self, x):
        # N x 3 x 224 x 224
        x = self.conv1(x)
        # N x 64 x 112 x 112
        x = self.maxpool1(x)
        # N x 64 x 56 x 56
        x = self.conv2(x)
        # N x 64 x 56 x 56
        x = self.conv3(x)
        # N x 192 x 56 x 56
        x = self.maxpool2(x)

        # N x 192 x 28 x 28
        x = self.inception3a(x)
        # N x 256 x 28 x 28
        x = self.inception3b(x)
        # N x 480 x 28 x 28
        x = self.maxpool3(x)
        # N x 480 x 14 x 14
        x = self.inception4a(x)
        # N x 512 x 14 x 14
        if self.training and self.aux_logits:    # eval model lose this layer
            # 判断当前是属于训练模式还是验证模式，训练则为true
            aux1 = self.aux1(x)

        x = self.inception4b(x)
        # N x 512 x 14 x 14
        x = self.inception4c(x)
        # N x 512 x 14 x 14
        x = self.inception4d(x)
        # N x 528 x 14 x 14
        if self.training and self.aux_logits:    # eval model lose this layer
            aux2 = self.aux2(x)

        x = self.inception4e(x)
        # N x 832 x 14 x 14
        x = self.maxpool4(x)
        # N x 832 x 7 x 7
        x = self.inception5a(x)
        # N x 832 x 7 x 7
        x = self.inception5b(x)
        # N x 1024 x 7 x 7

        x = self.avgpool(x)
        # N x 1024 x 1 x 1
        x = paddle.flatten(x, 1)
        # N x 1024
        x = self.dropout(x)
        x = self.fc(x)
        # N x 1000 (num_classes)
        if self.training and self.aux_logits:   # eval model lose this layer
            return x, aux2, aux1  # 训练模式返回3个值
        return x


class Inception(nn.Layer):
    def __init__(self, in_channels, ch1x1, ch3x3red, ch3x3, ch5x5red, ch5x5, pool_proj):
        # 主要有6个参数，第一个是输入深度
        # 分别对应表中的各结构
        super(Inception, self).__init__()

        self.branch1 = BasicConv2D(in_channels, ch1x1, kernel_size=1, weight_attr=nn.initializer.KaimingNormal())
        # 改变通道数

        self.branch2 = nn.Sequential(
            BasicConv2D(in_channels, ch3x3red, kernel_size=1, weight_attr=nn.initializer.KaimingNormal()),  # 降维
            BasicConv2D(ch3x3red, ch3x3, kernel_size=3, padding=1, weight_attr=nn.initializer.KaimingNormal())   # 保证输出大小等于输入大小
            # -3+2+1=0
        )
        # 必须保证每个branch的输出高宽是相同的


        self.branch3 = nn.Sequential(
            BasicConv2D(in_channels, ch5x5red, kernel_size=1, weight_attr=nn.initializer.KaimingNormal()),
            BasicConv2D(ch5x5red, ch5x5, kernel_size=5, padding=2, weight_attr=nn.initializer.KaimingNormal())   # 保证输出大小等于输入大小
            # -5+4+1=0
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool2D(kernel_size=3, stride=1, padding=1),  # 大小不变
            BasicConv2D(in_channels, pool_proj, kernel_size=1, weight_attr=nn.initializer.KaimingNormal())
        )


    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)
        # 分别输入到每个batch中

        outputs = [branch1, branch2, branch3, branch4]
        # 合并，长宽相同但是channel不一样
        return paddle.concat(outputs, 1)  # 深度上进行拼接


class InceptionAux(nn.Layer):  # 辅助分类器
    def __init__(self, in_channels, num_classes):  # 输入深度与类别个数
        super(InceptionAux, self).__init__()
        self.averagePool = nn.AvgPool2D(kernel_size=5, stride=3)  # 平均池化
        self.conv = BasicConv2D(in_channels, 128, kernel_size=1, weight_attr=nn.initializer.KaimingNormal())  # output[batch, 128, 4, 4]

        self.fc1 = nn.Linear(2048, 1024, weight_attr=nn.initializer.KaimingNormal())
        self.fc2 = nn.Linear(1024, num_classes, weight_attr=nn.initializer.KaimingNormal())
        # 全连接层

    def forward(self, x):
        # aux1: N x 512 x 14 x 14, aux2: N x 528 x 14 x 14
        x = self.averagePool(x)
        # aux1: N x 512 x 4 x 4, aux2: N x 528 x 4 x 4
        x = self.conv(x)
        # N x 128 x 4 x 4
        x = paddle.flatten(x, 1)
        # 展平处理
        x = F.dropout(x, 0.5, training=self.training)
        # 加入dropout函数
        # N x 2048
        x = F.relu(self.fc1(x))
        x = F.dropout(x, 0.5, training=self.training)
        # N x 1024
        x = self.fc2(x)
        # N x num_classes
        return x


class BasicConv2D(nn.Layer):
    def __init__(self, in_channels, out_channels, **kwargs):
        # 输入深度与输出深度
        super(BasicConv2D, self).__init__()
        self.conv = nn.Conv2D(in_channels, out_channels, **kwargs)
        self.relu = nn.ReLU()
        # conv, relu合在一起
    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        return x

In [4]:
import os
import json
import time

import paddle
import paddle.nn as nn
from paddle.vision import transforms
from paddle.vision import datasets
import paddle.optimizer as optim
from paddle.static import InputSpec

device = paddle.set_device("gpu")  # 指定设备
print("using {} device.".format(device))

data_transform = {
    # 训练集预处理
    "train": transforms.Compose([transforms.RandomResizedCrop(224),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),
    # 验证集预处理
    "val": transforms.Compose([transforms.Resize((224, 224)),  # cannot 224, must (224, 224)
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])}

data_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))  # get data root path
train_dataset = datasets.DatasetFolder(root="data/train",
                                     transform=data_transform["train"])
train_num = len(train_dataset)
garbage_list = train_dataset.class_to_idx
cla_dict = dict((val, key) for key, val in garbage_list.items())  


json_str = json.dumps(cla_dict, indent=4)
with open('class_indices.json', 'w') as json_file:
    json_file.write(json_str)



batch_size = 128
train_loader = paddle.io.DataLoader(train_dataset,
                                    batch_size=batch_size, shuffle=True,
                                    num_workers=0)
validate_dataset = datasets.DatasetFolder(root="data/val",
                                          transform=data_transform["val"])
val_num = len(validate_dataset)
validate_loader = paddle.io.DataLoader(validate_dataset,
                                        batch_size=batch_size, shuffle=True,
                                        num_workers=0)
print("using {} images for training, {} images for validation.".format(train_num,
                                                                       val_num))



net = GoogLeNet(num_classes=40, aux_logits=True)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(parameters=net.parameters(), learning_rate=0.0003)



using CUDAPlace(0) device.
using 12980 images for training, 1422 images for validation.


请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

In [6]:
epochs = 200
best_acc = 0.0
train_steps = len(train_loader)
for epoch in range(epochs):
    # train
    net.train()  # 启用dropout
    running_loss = 0.0
    t1 = time.perf_counter()
    for step, data in enumerate(train_loader, start=0):
        images, labels = data
        logits, aux_logits2, aux_logits1 = net(images)
        loss0 = loss_function(logits, labels)  # 主分类器
        loss1 = loss_function(aux_logits1, labels)  # 辅助
        loss2 = loss_function(aux_logits2, labels)
        loss = loss0 + loss1 * 0.3 + loss2 * 0.3
        # 有三个输出
        loss.backward()
        optimizer.step()
        optimizer.clear_grad() # 梯度置零

        # print statistics
        running_loss += loss.numpy()[0]
        rate = (step + 1) / len(train_loader)
        a = "*" * int(rate * 50)
        b = "." * int((1 - rate) * 50)
        loss_num = loss.numpy()[0]
        print("\rtrain loss: {:^3.0f}%[{}->{}]{:.3f}".format(int(rate * 100), a, b, loss_num), end=" ")
    print()
    print(time.perf_counter()-t1)
    
    # validate
    net.eval()  # 关闭dropout
    acc = 0.0  # accumulate accurate number / epoch
    with paddle.no_grad():  # 禁止跟踪
        for val_data in validate_loader:
            val_images, val_labels = val_data
            outputs = net(val_images)
            predict_y = outputs.numpy().argmax(axis=1)
            acc += (val_labels.astype("int").numpy()==outputs.numpy().argmax(axis=1).astype("int")).sum()

        val_accurate = acc / val_num
        print('[epoch %d] train_loss: %.3f  val_accuracy: %.3f' %
              (epoch + 1, running_loss / train_steps, val_accurate))

        if val_accurate > best_acc:
            paddle.jit.save(
                layer=net,
                path="result",
                input_spec=[InputSpec(shape=[1, 3, 224, 224], dtype='float32')])

print('Finished Training')





train loss: 100%[**************************************************->]5.457 
96.86560590099543
[epoch 1] train_loss: 5.651  val_accuracy: 0.109


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(seq, collections.Sequence) and


train loss: 100%[**************************************************->]4.880 
100.93733469303697
[epoch 2] train_loss: 5.221  val_accuracy: 0.158
train loss: 100%[**************************************************->]5.142 
100.60623106593266
[epoch 3] train_loss: 5.022  val_accuracy: 0.184
train loss: 100%[**************************************************->]4.840 
101.89062007516623
[epoch 4] train_loss: 4.871  val_accuracy: 0.185
train loss: 100%[**************************************************->]4.712 
100.24071261403151
[epoch 5] train_loss: 4.741  val_accuracy: 0.231
train loss: 100%[**************************************************->]4.507 
100.9687779659871
[epoch 6] train_loss: 4.601  val_accuracy: 0.205
train loss: 100%[**************************************************->]4.360 
100.05084817484021
[epoch 7] train_loss: 4.484  val_accuracy: 0.243
train loss: 100%[**************************************************->]4.591 
100.16725850501098
[epoch 8] train_loss: 4.360  val_ac

In [7]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

with zipfile.ZipFile("data/data35095/test.zip") as zf:
   zf.extractall()

data_transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# load image
with open("test.txt","w") as f:
    for i in range(1,401):
        img_path = "test/test"+str(i)+".jpg"
        assert os.path.exists(img_path), "file: '{}' dose not exist.".format(img_path)
        img = Image.open(img_path)

        # plt.imshow(img)
        # [N, C, H, W]
        img = data_transform(img)
        # expand batch dimension
        img = paddle.unsqueeze(img, axis=0)
        # read class_indict
        json_path = 'class_indices.json'
        assert os.path.exists(json_path), "file: '{}' dose not exist.".format(json_path)

        json_file = open(json_path, "r")
        class_indict = json.load(json_file)

        # create model
        model = GoogLeNet(num_classes=40, aux_logits=False)

        # load model weights
        # weights_path = "AlexNet.pth"
        # assert os.path.exists(weights_path), "file: '{}' dose not exist.".format(weights_path)
        model = paddle.jit.load("result")
        # model.load_state_dict(torch.load(weights_path))

        model.eval()
        with paddle.no_grad():
            # predict class
            output = paddle.squeeze(model(img)[0])
            predict = nn.Softmax(axis=0)(output)
            predict_cla = paddle.argmax(predict).numpy()[0]

        # print_res = "class: {}   prob: {:.3}".format(class_indict[str(predict_cla)],
                                                    #predict.numpy()[predict_cla])
        # print(print_res)
    
        f.write(class_indict[str(predict_cla)]+"\n")

        rateTest = i / 400
        aTest = "*" * int(rateTest * 50)
        bTest = "." * int((1 - rateTest) * 50)
        print("\rprocess: {:d}% [{}->{}]".format(int(rateTest * 100), aTest, bTest), end=" ")

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized
2021-03-31 22:35:04,775 - INFO - font search path ['/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf', '/opt/co

fail to perform transform [<paddle.vision.transforms.transforms.ToTensor object at 0x7fe8fe0ac390>] with error: We only support 'to_tensor()' in dynamic graph mode, please call 'paddle.disable_static()' to enter dynamic graph mode. and stack:
Traceback (most recent call last):
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/vision/transforms/transforms.py", line 117, in __call__
    data = f(data)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/vision/transforms/transforms.py", line 273, in __call__
    outputs.append(apply_func(inputs[i]))
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/vision/transforms/transforms.py", line 347, in _apply_image
    return F.to_tensor(img, self.data_format)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/vision/transforms/functional.py", line 91, in to_tensor
    return F_pil.to_tensor(pic, data_format)
  File "/opt/c

Exception in thread Thread-6:
Traceback (most recent call last):
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py", line 346, in _thread_loop
    six.reraise(*sys.exc_info())
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/six.py", line 703, in reraise
    raise value
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py", line 317, in _thread_loop
    batch = self._dataset_fetcher.fetch(indices)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/fetcher.py", line 60, in fetch
    data = [self.dataset[idx] for idx in batch_ind

process: 100% [**************************************************->] 